# Part 3 (Extra credit 5 points)

ถ้าเราจะเลือกเพียงแค่ 3 Feature มาใช้ในการทำ Machine Learning (เพื่อเพิ่ม Efficiency ในการ Training) ในกรนีนี้เราควรจะเลือก Feature อะไรที่ให้ Accuracy มากที่สุด

## Algorithm

ในกรณีนี้เราจะใช้ K-Nearest Neighbor ในการช่วยตัดสินใจในการเลือก Feature (แต่ในการทำ Prediction จริงเราอาจจะใช้ตัวอื่นได้)

1. เลือก Feature ขึ้นมา 3 Feature
2. ใช้ K-Nearest Neighbor ในการ Train ข้อมูล Training Data
3. หา Accuracy โดยการใช้ Test Data

ทำข้อ 1. ถึง 3. กับทุก Combination ของ 3 Feature ที่จะเป็นไปได้ แล้วคำตอบก็คือ ชุด Feature ที่มีค่า Accuracy มากที่สุด 

## Tips

- เวลาตอบให้เขียน Code ที่แสดงผลชื่อ Column ที่เลือก และ Accuracy เช่น
    - Alcohol, Malic acid, Ash (Accuracy = 0.88) (Format ไม่ต้องตามนี้ก็ได้ แต่ข้อมูลขอแบบนี้)
- คำตอบอาจจะมีได้มากกว่า 1 คำตอบ
- อย่าลืมว่า K-Nearest Neighbor ต้องทำการ Standardize ข้อมูล
- สามารถใช้ itertools.combinations ในการสร้าง Unique Set ของ Feature ที่จะทดลอง (ใช้วิธีอื่นก็ได้)

```python
import itertools
all_combos = list(itertools.combinations([1,2,3,5], 2))
print(all_combos)
#[(1, 2), (1, 3), (1, 5), (2, 3), (2, 5), (3, 5)]

```

In [38]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import itertools


In [39]:
model_names = ['Perceptron', 'LogisticRegression',
               'SVC', 'DecisionTree', 'RandomForrest', 'KNN', 'NaiveBayes']

# Read data
df = pd.read_csv("wine_data.csv", header=None)
df.columns = ["Class label", "Alcohol", "Malic acid", "Ash", "Alcalinity of ash", "Magnesium", "Total phenols", "Flavanoids",
              "Nonflavanoid phenols", "Proanthocyanins", "Color intensity", "Hue", "OD280/OD315 of diluted wines", "Proline"]

# Extract values
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

# Get X column name
colsX = df.iloc[:, 1:].columns.values

# Split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

# Standardization
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [40]:
numCol = 3
colIdxs = [i for i in range(X_train_std.shape[1])]
colCombos = list(itertools.combinations(colIdxs, numCol))

In [41]:
colX = df.iloc[:, 1:].columns
dataArr = []
for idxs in colCombos:

    X_train_std_reduced = X_train_std[:, idxs]
    X_test_std_reduce = X_test_std[:, idxs]

    model = KNeighborsClassifier(n_neighbors=5)

    model.fit(X_train_std_reduced, y_train)

    # Prediction
    y_pred = model.predict(X_test_std_reduce)

    # Accuracy score from the test samples
    acc = accuracy_score(y_test, y_pred)

    # Create column names
    colsArr = [f'{i}-{colsX[i]}'[:7] for i in idxs]
    colsArrPrint = ', '.join(colsArr)

    # Store data
    data = {
        'idxs': idxs,
        'acc': acc,
        'cols': colsArrPrint,
    }
    dataArr.append(data)


In [42]:
df = pd.DataFrame.from_records(dataArr)
display(df)

,idxs,acc,cols
0,"(0, 1, 2)",0.759259,"0-Alcoh, 1-Malic, 2-Ash"
1,"(0, 1, 3)",0.722222,"0-Alcoh, 1-Malic, 3-Alcal"
2,"(0, 1, 4)",0.740741,"0-Alcoh, 1-Malic, 4-Magne"
3,"(0, 1, 5)",0.851852,"0-Alcoh, 1-Malic, 5-Total"
4,"(0, 1, 6)",0.907407,"0-Alcoh, 1-Malic, 6-Flava"
...,...,...,...
281,"(8, 11, 12)",0.851852,"8-Proan, 11-OD28, 12-Prol"
282,"(9, 10, 11)",0.888889,"9-Color, 10-Hue, 11-OD28"
283,"(9, 10, 12)",0.888889,"9-Color, 10-Hue, 12-Prol"
284,"(9, 11, 12)",0.907407,"9-Color, 11-OD28, 12-Prol"


In [43]:
df.sort_values(by='acc', ascending=False).head(50)

,idxs,acc,cols
211,"(4, 6, 9)",0.944444,"4-Magne, 6-Flava, 9-Color"
64,"(0, 10, 12)",0.944444,"0-Alcoh, 10-Hue, 12-Prol"
277,"(8, 9, 11)",0.944444,"8-Proan, 9-Color, 11-OD28"
31,"(0, 4, 6)",0.925926,"0-Alcoh, 4-Magne, 6-Flava"
262,"(6, 9, 12)",0.925926,"6-Flava, 9-Color, 12-Prol"
53,"(0, 7, 10)",0.925926,"0-Alcoh, 7-Nonfl, 10-Hue"
102,"(1, 6, 9)",0.925926,"1-Malic, 6-Flava, 9-Color"
23,"(0, 3, 6)",0.925926,"0-Alcoh, 3-Alcal, 6-Flava"
115,"(1, 9, 10)",0.925926,"1-Malic, 9-Color, 10-Hue"
116,"(1, 9, 11)",0.925926,"1-Malic, 9-Color, 11-OD28"
